# Export Transactions All
and create xlsx-Files

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../data/lib/')
import consts
from pdfexport import *
import openpyxl
import subprocess, os, sys
import mysql.connector

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [3]:
df_recipient = pd.read_sql("""SELECT 
      tra_fk_recipient as id,
      max(recipient.rec_name) as name,
      max(recipient.rec_address) as address,
      max(recipient.rec_location) as location,
      max(recipient.rec_plz) as plz,
      max(recipient.rec_type) as type,
      max(tra_year) as maxYear
    FROM transaction
    LEFT JOIN recipient ON rec_id = transaction.tra_fk_recipient
    GROUP BY tra_fk_recipient""", con=db_connection)

df_transaction = pd.read_sql("""SELECT 
    max(tra_fk_recipient) as id,
    max(tra_name_original) as name,
    max(tra_location_original) as location,
    max(tra_address_original) as address,
    max(tra_year) as maxYear,
    max(recipient.rec_type) as type
  FROM transaction LEFT JOIN recipient ON rec_id = tra_fk_recipient
  GROUP BY tra_name_original, tra_location_original, tra_address_original""", con=db_connection)

## Export

In [4]:
df_recipient['mainaddress'] = True
df_transaction['mainaddress'] = False

#Concat
df_concat = pd.concat([df_recipient, df_transaction], sort=False)
df_concat = df_concat.sort_values(['id', 'mainaddress'], ascending = [True, False])

df_concat = df_concat.reset_index(drop=False)

df_hcp = df_concat[df_concat.type == 'hcp']
df_hco = df_concat[df_concat.type == 'hco']

df_hcp = df_hcp.set_index(['id', 'index'])
df_hco = df_hco.set_index(['id', 'index'])

#Write all
writer = pd.ExcelWriter('../../data/99. analyzes/excel/all_transactions.xlsx', options={'encoding':'utf-8'})
df_hcp.to_excel(writer, 'hcp', index=True)
df_hco.to_excel(writer, 'hco', index=True)
writer.save()